In [ ]:
# Corriger les incompatibilités de versions
!pip install --upgrade --force-reinstall numpy==1.23.5 gensim==4.3.0 scipy==1.10.1 --quiet

# Redémarrer automatiquement l'environnement
import os
os.kill(os.getpid(), 9)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
t

Installer nltk

Importer les bibliothèques

In [ ]:
!pip install gensim nltk

In [ ]:
# Import des bibliothèques de base
import os
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import defaultdict
from time import time

Charger mon drive dans google collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


chargement des Données

In [ ]:
# Chemin vers le dossier de données
data_dir = "/content/drive/MyDrive/Parlamint2018_raw"

# Chargement des textes
documents = []
labels = []

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".txt"):
            with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                text = f.read().strip()
                documents.append(text)
                labels.append(os.path.basename(root))  # Utilise le nom du dossier comme label

print(f"✅ {len(documents)} documents chargés.")
print(f"✅ {len(set(labels))} classes détectées.")

✅ 362 documents chargés.
✅ 362 classes détectées.


Aperçu Des Données

In [ ]:
# Aperçu d'un texte et des labels
print("📄 Exemple de texte (500 premiers caractères) :\n")
print(documents[0][:500])

print("\n Labels détectés :")
print(sorted(set(labels))[:10])  # Affiche les 10 premiers labels pour un aperçu
print(f"\n Nombre total de labels : {len(set(labels))}")


📄 Exemple de texte (500 premiers caractères) :

Mes chers collègues, avant d’en venir aux questions au Gouvernement, je voudrais faire une petite mise au point. Tous les groupes politiques représentés dans notre assemblée se sont joints la semaine dernière, dans une démarche commune, à la marche blanche organisée mercredi en hommage à Mireille Knoll et en signe de refus de l’antisémitisme. Nos travaux ont d’ailleurs été suspendus pour pouvoir aller à cette manifestation. Je veux ici, en votre nom, comme je l’avais fait publiquement dès mercre

 Labels détectés :
['ParlaMint-FR_2018-01-16-O1111', 'ParlaMint-FR_2018-01-16-O1112', 'ParlaMint-FR_2018-01-17-O1113', 'ParlaMint-FR_2018-01-17-O1114', 'ParlaMint-FR_2018-01-18-O1115', 'ParlaMint-FR_2018-01-18-O1116', 'ParlaMint-FR_2018-01-18-O1117', 'ParlaMint-FR_2018-01-22-O1118', 'ParlaMint-FR_2018-01-23-O1119', 'ParlaMint-FR_2018-01-23-O1120']

 Nombre total de labels : 362


Extraction des textes

In [ ]:
import os
import pandas as pd

# Chemin vers le dossier contenant les fichiers .txt
data_dir = "/content/drive/MyDrive/Parlamint2018_raw"

documents = []
labels = []

# Parcours des sous-dossiers
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    text = f.read().strip()
                    documents.append(str(text))
                    labels.append(str(os.path.basename(root)))
            except Exception as e:
                print(f"❌ Erreur avec le fichier {file_path} : {e}")

# Vérification
print(f"✅ Nombre de documents : {len(documents)}")
print(f"✅ Nombre de labels : {len(labels)}")

print(f"Type des éléments de documents : {[type(doc) for doc in documents[:5]]}")
print(f"Type des éléments de labels : {[type(label) for label in labels[:5]]}")


#Forcer la convertion en string
documents = [str(doc) for doc in documents]
labels = [str(label) for label in labels]


✅ Nombre de documents : 362
✅ Nombre de labels : 362
Type des éléments de documents : [<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]
Type des éléments de labels : [<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]


Vérification

In [ ]:
for i, doc in enumerate(documents):
    if not isinstance(doc, str):
        print(f"❌ Problème dans documents à l'indice {i}: {type(doc)}")

for i, lab in enumerate(labels):
    if not isinstance(lab, str):
        print(f"❌ Problème dans labels à l'indice {i}: {type(lab)}")


Créer le dataframe

In [ ]:
import pandas as pd

# Nettoyage explicite : filtrer tout ce qui n'est pas une string
documents_clean = [str(doc) if not isinstance(doc, str) else doc for doc in documents]
labels_clean = [str(label) if not isinstance(label, str) else label for label in labels]

# Conversion explicite en listes standard (pour éviter les objets NumPy)
documents_clean = list(documents_clean)
labels_clean = list(labels_clean)

# Création du DataFrame
df_texts = pd.DataFrame({
    "label": pd.Series(labels_clean, dtype="string"),
    "text": pd.Series(documents_clean, dtype="string")
})

print("✅ DataFrame créé avec succès.")
display(df_texts.sample(3))


TypeError: Cannot convert numpy.ndarray to numpy.ndarray

Installer le modèle Spacy Français

In [ ]:
# Installer Spacy Français
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 88.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Installer & Charger SpaCy

In [ ]:
import re
import spacy

# Charger le modèle SpaCy pour le français
nlp = spacy.load("fr_core_news_sm", disable=["ner", "parser"])

Nettoyage des Textes

In [ ]:
def nettoyage_texte(texte):
    # Nettoyage de base (garde seulement les lettres et apostrophes)
    texte = re.sub(r"[^a-zA-ZÀ-ÿ']+", ' ', texte)
    # Lemmatisation et suppression des stopwords
    doc = nlp(texte)
    return [token.lemma_ for token in doc if not token.is_stop and len(token) > 2]

# Application du nettoyage sur les textes
df_texts["tokens"] = df_texts["text"].apply(nettoyage_texte)

print(f"✅ {len(df_texts)} documents nettoyés.")
print(f"📝 Exemple de tokens : {df_texts['tokens'].iloc[0][:20]}")


NameError: name 'df_texts' is not defined

Sauvegarder les tockens néttoyés dans un csv

In [ ]:
# Sauvegarde des tokens nettoyés dans un fichier CSV
df_texts.to_csv("documents_nettoyes.csv", index=False)

print("✅ Fichier 'documents_nettoyes.csv' sauvegardé.")


Chargement du CSV

In [ ]:
import pandas as pd
import ast

# Charger le fichier CSV
df_texts = pd.read_csv("documents_nettoyes.csv")

# Convertir la colonne "tokens" en liste Python
df_texts["tokens"] = df_texts["tokens"].apply(ast.literal_eval)


FileNotFoundError: [Errno 2] No such file or directory: 'documents_nettoyes.csv'

Création des bigrammes

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# Détection des bigrammes
phrases = Phrases(df_texts["tokens"], min_count=5, threshold=10)
bigram = Phraser(phrases)

# Transformation des documents avec les bigrammes
df_texts["tokens_bigrams"] = df_texts["tokens"].apply(lambda x: bigram[x])

print(f"✅ Bigrams créés. Exemple : {df_texts['tokens_bigrams'].iloc[0][:20]}")


Entrainement du modèle

In [ ]:
import multiprocessing
from gensim.models import Word2Vec

# Préparation des phrases pour l'entraînement
sentences = df_texts["tokens_bigrams"].tolist()


# Paramètres du modèle
w2v_model = Word2Vec(
    sentences,
    vector_size=100,  # Taille des vecteurs
    window=5,  # Contexte
    min_count=5,  # Fréquence minimale
    workers=multiprocessing.cpu_count(),  # Utilise tous les cœurs disponibles
    sg=1  # 1 pour skip-gram (meilleure précision), 0 pour CBOW (plus rapide)
)

# Entraîner le modèle
w2v_model.train(sentences, total_examples=len(sentences), epochs=10)

print("✅ Modèle Word2Vec entraîné.")


Vérifier la similarité

In [ ]:
# Exemples de mots courants dans le corpus
exemples = ["gouvernement", "président", "ministre", "démocratie", "loi"]

for mot in exemples:
    try:
        similaires = w2v_model.wv.most_similar(positive=[mot], topn=10)
        print(f"\n🔹 Mots les plus similaires à '{mot}' :")
        for sim_mot, score in similaires:
            print(f"   {sim_mot} (similarité : {score:.2f})")
    except KeyError:
        print(f"⚠️ Le mot '{mot}' n'est pas dans le vocabulaire du modèle.")


Calculer la similarité entre deux mots " gouvernement" et "ministre"

In [ ]:
# Vérifie la similarité entre deux mots
mot1 = "gouvernement"
mot2 = "ministre"

try:
    similarite = w2v_model.wv.similarity(mot1, mot2)
    print(f"\n🔹 Similarité entre '{mot1}' et '{mot2}' : {similarite:.2f}")
except KeyError as e:
    print(f"⚠️ {e}")


***Clustering***:

1.   Préparer les vecteurs Word2Vec Pour WARD



In [ ]:
import numpy as np

# Récupérer tous les mots du modèle
words = list(w2v_model.wv.index_to_key)

# Extraire leurs vecteurs
X = np.array([w2v_model.wv[word] for word in words])

print(f"✅ {len(words)} mots extraits pour le clustering.")

In [ ]:
print(words[:5])



2.   Réduction de dimension( obligatoire pour WARD)
L'algorithme de Ward nécessite des données en espace euclidien, donc il faut réduire la dimension avec PCA avant de l'utiliser.






In [ ]:
from sklearn.decomposition import PCA

# Réduction de la dimension à 20 pour limiter l'utilisation de la mémoire
pca = PCA(n_components=20, random_state=42)
X_reduced = pca.fit_transform(X)

print(f"✅ Dimensions réduites à : {X_reduced.shape}")


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Étape 1 : PCA vers 10 dimensions (rapide)
X_pca_10 = PCA(n_components=10, random_state=42).fit_transform(X_reduced)

# Étape 2 : t-SNE 2D (plus rapide après réduction)
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
X_2d = tsne.fit_transform(X_pca_10)

print(f"✅ Réduction t-SNE terminée. Dimensions : {X_2d.shape}")




Clustering avec WARD


In [ ]:
from sklearn.cluster import AgglomerativeClustering

n_clusters = 10
ward = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward")
labels = ward.fit_predict(X_reduced)

print(f"✅ Clustering terminé avec {n_clusters} clusters.")

Visualisation